# DATA70121 Coursework

【说明】导入数值计算、统计建模与可视化相关的全部库，支撑报告的方法论与附录代码运行环境。

In [334]:
# Load the libraries we'll use ######################################
# Basic numerics
import numpy as np
import scipy.stats as stats

# Suppress certain irrelevant warnings in the graphics libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Graphics
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

# Data handling
import pandas as pd
from pathlib import Path

# 设置全局随机种子以确保结果可重复
import random
np.random.seed(123)
random.seed(123)


Before doing anything else, we make sure there's a directory to receive the figures we'll plot.

【说明】创建并确认 `Figures` 目录，保证所有图像以 PDF 保存，方便在报告和附录中统一引用。

In [335]:

# https://www.tutorialspoint.com/How-can-I-create-a-directory-if-it-does-not-exist-using-Python
FIG_DIR = Path('Figures')
if not FIG_DIR.exists():
    FIG_DIR.mkdir(parents=True)
print(f"Figures directory: {FIG_DIR.resolve()}")


Figures directory: /Users/vendredi/Library/Mobile Documents/com~apple~CloudDocs/UoManchester/DATA70121_Stat_ML/DATA70121_Coursework/Figures


## Reading the data

【说明】读取 MavenRail 数据并展示前几行，为报告的数据来源与初步描述提供直接证据。

In [336]:

data_path = Path('MavenRail_cleaned2.csv')
assert data_path.exists(), 'MavenRail_cleaned2.csv 未找到，请确认路径'
rail_df = pd.read_csv(data_path)
rail_df.head()


,Payment.Method,Railcard,Ticket.Class,Ticket.Type,Price,Departure.Station,Arrival.Station,Departure,Scheduled.Arrival,Actual.Arrival,Journey.Status,Reason.for.Delay,Refund.Request
0,Contactless,Adult,Standard,Advance,43,London Paddington,Liverpool Lime Street,2024-01-01 11:00,2024-01-01 13:30,2024-01-01 13:30,On Time,NaN,No
1,Credit Card,Adult,Standard,Advance,23,London Kings Cross,York,2024-01-01 09:45,2024-01-01 11:35,2024-01-01 11:40,Delayed,Signal Failure,No
2,Credit Card,NaN,Standard,Advance,3,Liverpool Lime Street,Manchester Piccadilly,2024-01-02 18:15,2024-01-02 18:45,2024-01-02 18:45,On Time,NaN,No
3,Credit Card,NaN,Standard,Advance,13,London Paddington,Reading,2024-01-01 21:30,2024-01-01 22:30,2024-01-01 22:30,On Time,NaN,No
4,Contactless,NaN,Standard,Advance,76,Liverpool Lime Street,London Euston,2024-01-01 16:45,2024-01-01 19:00,2024-01-01 19:00,On Time,NaN,No


## 全局数据质量检查与基础清洗

【说明】在执行各题分析之前，对全表进行一次基础质量检查（重复行、关键字段缺失、时间列可转换性），确保后续派生变量与模型建立在干净的数据上。

#### 重复记录检测

【说明】统计并移除完全重复的记录，避免在后续分析中重复计算同一旅程。

In [337]:
duplicate_mask = rail_df.duplicated(keep=False)
duplicate_count = rail_df.duplicated().sum()
print(f"检测到的完全重复行数：{duplicate_count}")

if duplicate_count > 0:
    print("重复组示例（含首行及其余重复行）：")
    dup_groups = (
        rail_df[duplicate_mask]
        .groupby(rail_df.columns.tolist(), dropna=False)
        .size()
        .reset_index(name='count')
        .sort_values('count', ascending=False)
        .head(3)   # 按需调整示例组数
    )

    placeholder = '<<NA>>'  # 用于把 NaN 视作“相等”
    rail_df_filled = rail_df.fillna(placeholder)

    for i in range(len(dup_groups)):
        combo = dup_groups.iloc[i, :-1]
        count = dup_groups.iloc[i, -1]
        print(f"\n组合 {i+1}：出现 {count} 次")
        combo_filled = combo.fillna(placeholder)
        mask = rail_df_filled.eq(combo_filled).all(axis=1)
        display(rail_df[mask])
else:
    print("未发现完全重复的记录。")


检测到的完全重复行数：5525
重复组示例（含首行及其余重复行）：

组合 1：出现 11 次


,Payment.Method,Railcard,Ticket.Class,Ticket.Type,Price,Departure.Station,Arrival.Station,Departure,Scheduled.Arrival,Actual.Arrival,Journey.Status,Reason.for.Delay,Refund.Request
8102,Credit Card,NaN,Standard,Anytime,16,London St Pancras,Birmingham New Street,2024-01-31 18:45,2024-01-31 20:05,2024-01-31 20:05,On Time,NaN,No
8108,Credit Card,NaN,Standard,Anytime,16,London St Pancras,Birmingham New Street,2024-01-31 18:45,2024-01-31 20:05,2024-01-31 20:05,On Time,NaN,No
8109,Credit Card,NaN,Standard,Anytime,16,London St Pancras,Birmingham New Street,2024-01-31 18:45,2024-01-31 20:05,2024-01-31 20:05,On Time,NaN,No
8110,Credit Card,NaN,Standard,Anytime,16,London St Pancras,Birmingham New Street,2024-01-31 18:45,2024-01-31 20:05,2024-01-31 20:05,On Time,NaN,No
8111,Credit Card,NaN,Standard,Anytime,16,London St Pancras,Birmingham New Street,2024-01-31 18:45,2024-01-31 20:05,2024-01-31 20:05,On Time,NaN,No
8113,Credit Card,NaN,Standard,Anytime,16,London St Pancras,Birmingham New Street,2024-01-31 18:45,2024-01-31 20:05,2024-01-31 20:05,On Time,NaN,No
8116,Credit Card,NaN,Standard,Anytime,16,London St Pancras,Birmingham New Street,2024-01-31 18:45,2024-01-31 20:05,2024-01-31 20:05,On Time,NaN,No
8117,Credit Card,NaN,Standard,Anytime,16,London St Pancras,Birmingham New Street,2024-01-31 18:45,2024-01-31 20:05,2024-01-31 20:05,On Time,NaN,No
8118,Credit Card,NaN,Standard,Anytime,16,London St Pancras,Birmingham New Street,2024-01-31 18:45,2024-01-31 20:05,2024-01-31 20:05,On Time,NaN,No
8119,Credit Card,NaN,Standard,Anytime,16,London St Pancras,Birmingham New Street,2024-01-31 18:45,2024-01-31 20:05,2024-01-31 20:05,On Time,NaN,No



组合 2：出现 10 次


,Payment.Method,Railcard,Ticket.Class,Ticket.Type,Price,Departure.Station,Arrival.Station,Departure,Scheduled.Arrival,Actual.Arrival,Journey.Status,Reason.for.Delay,Refund.Request
4671,Credit Card,NaN,Standard,Anytime,151,Liverpool Lime Street,London Euston,2024-01-19 08:00,2024-01-19 10:15,2024-01-19 11:13,Delayed,Weather,No
4672,Credit Card,NaN,Standard,Anytime,151,Liverpool Lime Street,London Euston,2024-01-19 08:00,2024-01-19 10:15,2024-01-19 11:13,Delayed,Weather,No
4673,Credit Card,NaN,Standard,Anytime,151,Liverpool Lime Street,London Euston,2024-01-19 08:00,2024-01-19 10:15,2024-01-19 11:13,Delayed,Weather,No
4675,Credit Card,NaN,Standard,Anytime,151,Liverpool Lime Street,London Euston,2024-01-19 08:00,2024-01-19 10:15,2024-01-19 11:13,Delayed,Weather,No
4677,Credit Card,NaN,Standard,Anytime,151,Liverpool Lime Street,London Euston,2024-01-19 08:00,2024-01-19 10:15,2024-01-19 11:13,Delayed,Weather,No
4678,Credit Card,NaN,Standard,Anytime,151,Liverpool Lime Street,London Euston,2024-01-19 08:00,2024-01-19 10:15,2024-01-19 11:13,Delayed,Weather,No
4679,Credit Card,NaN,Standard,Anytime,151,Liverpool Lime Street,London Euston,2024-01-19 08:00,2024-01-19 10:15,2024-01-19 11:13,Delayed,Weather,No
4682,Credit Card,NaN,Standard,Anytime,151,Liverpool Lime Street,London Euston,2024-01-19 08:00,2024-01-19 10:15,2024-01-19 11:13,Delayed,Weather,No
4683,Credit Card,NaN,Standard,Anytime,151,Liverpool Lime Street,London Euston,2024-01-19 08:00,2024-01-19 10:15,2024-01-19 11:13,Delayed,Weather,No
4685,Credit Card,NaN,Standard,Anytime,151,Liverpool Lime Street,London Euston,2024-01-19 08:00,2024-01-19 10:15,2024-01-19 11:13,Delayed,Weather,No



组合 3：出现 9 次


,Payment.Method,Railcard,Ticket.Class,Ticket.Type,Price,Departure.Station,Arrival.Station,Departure,Scheduled.Arrival,Actual.Arrival,Journey.Status,Reason.for.Delay,Refund.Request
9376,Credit Card,NaN,Standard,Advance,8,London St Pancras,Birmingham New Street,2024-02-25 18:45,2024-02-25 20:05,2024-02-25 20:05,On Time,NaN,No
10190,Credit Card,NaN,Standard,Advance,8,London St Pancras,Birmingham New Street,2024-02-25 18:45,2024-02-25 20:05,2024-02-25 20:05,On Time,NaN,No
10577,Credit Card,NaN,Standard,Advance,8,London St Pancras,Birmingham New Street,2024-02-25 18:45,2024-02-25 20:05,2024-02-25 20:05,On Time,NaN,No
12714,Credit Card,NaN,Standard,Advance,8,London St Pancras,Birmingham New Street,2024-02-25 18:45,2024-02-25 20:05,2024-02-25 20:05,On Time,NaN,No
13232,Credit Card,NaN,Standard,Advance,8,London St Pancras,Birmingham New Street,2024-02-25 18:45,2024-02-25 20:05,2024-02-25 20:05,On Time,NaN,No
14264,Credit Card,NaN,Standard,Advance,8,London St Pancras,Birmingham New Street,2024-02-25 18:45,2024-02-25 20:05,2024-02-25 20:05,On Time,NaN,No
14592,Credit Card,NaN,Standard,Advance,8,London St Pancras,Birmingham New Street,2024-02-25 18:45,2024-02-25 20:05,2024-02-25 20:05,On Time,NaN,No
14597,Credit Card,NaN,Standard,Advance,8,London St Pancras,Birmingham New Street,2024-02-25 18:45,2024-02-25 20:05,2024-02-25 20:05,On Time,NaN,No
15089,Credit Card,NaN,Standard,Advance,8,London St Pancras,Birmingham New Street,2024-02-25 18:45,2024-02-25 20:05,2024-02-25 20:05,On Time,NaN,No


#### 关键字段缺失情况

【说明】检查票价、行程状态、退款标记以及三列时间字段是否存在缺失，如有缺失则仅对这些关键列做最小化清洗。

In [338]:

critical_cols = ['Price', 'Journey.Status', 'Refund.Request', 'Departure', 'Scheduled.Arrival', 'Actual.Arrival']
missing_summary = rail_df[critical_cols].isna().sum().to_frame(name='missing_count')
missing_summary['missing_pct'] = (missing_summary['missing_count'] / len(rail_df) * 100).round(2)
print(missing_summary)

# 仅对票价/行程状态/退款请求缺失的行进行剔除，其余时间字段在转换时再处理
rows_before = len(rail_df)
rail_df = rail_df.dropna(subset=['Price', 'Journey.Status', 'Refund.Request'])
print(f"删除关键分类字段缺失行 {rows_before - len(rail_df)} 条，剩余 {len(rail_df):,} 行。")


                   missing_count  missing_pct
Price                          0         0.00
Journey.Status                 0         0.00
Refund.Request                 0         0.00
Departure                      3         0.01
Scheduled.Arrival              4         0.01
Actual.Arrival              1835         5.98
删除关键分类字段缺失行 0 条，剩余 30,686 行。


#### 时间字段类型检查

【说明】确认三列时间字段当前的数据类型及样例值，为之后的 `pd.to_datetime` 转换提供依据，并记录无法解析的情况。

In [339]:

time_cols = ['Departure', 'Scheduled.Arrival', 'Actual.Arrival']
time_type_report = {col: rail_df[col].dtype for col in time_cols}
print('当前时间列类型：', time_type_report)

sample_times = rail_df[time_cols].head(3)
print("示例时间记录：")
print(sample_times)


当前时间列类型： {'Departure': dtype('O'), 'Scheduled.Arrival': dtype('O'), 'Actual.Arrival': dtype('O')}
示例时间记录：
          Departure Scheduled.Arrival    Actual.Arrival
0  2024-01-01 11:00  2024-01-01 13:30  2024-01-01 13:30
1  2024-01-01 09:45  2024-01-01 11:35  2024-01-01 11:40
2  2024-01-02 18:15  2024-01-02 18:45  2024-01-02 18:45


### Initial structure

【说明】调用 `info()` 查看每列的数据类型与缺失情况，可用于报告的数据概览与质量说明。

In [340]:
rail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30686 entries, 0 to 30685
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Payment.Method     30686 non-null  object
 1   Railcard           10359 non-null  object
 2   Ticket.Class       30686 non-null  object
 3   Ticket.Type        30686 non-null  object
 4   Price              30686 non-null  int64 
 5   Departure.Station  30686 non-null  object
 6   Arrival.Station    30686 non-null  object
 7   Departure          30683 non-null  object
 8   Scheduled.Arrival  30682 non-null  object
 9   Actual.Arrival     28851 non-null  object
 10  Journey.Status     30686 non-null  object
 11  Reason.for.Delay   4121 non-null   object
 12  Refund.Request     30686 non-null  object
dtypes: int64(1), object(12)
memory usage: 3.0+ MB


### Convert the date-time columns

【说明】把出发与到达时间字段统一转换为 `datetime`，为延误计算与时间特征分析做准备，可写入方法章节。

In [341]:

# Convert the dates into DateTime objects  
rail_df["Departure"] = pd.to_datetime( rail_df["Departure"], format='%Y-%m-%d %H:%M' )
rail_df["Scheduled.Arrival"] = pd.to_datetime( rail_df["Scheduled.Arrival"], format='%Y-%m-%d %H:%M' )
rail_df["Actual.Arrival"] = pd.to_datetime( rail_df["Actual.Arrival"], format='%Y-%m-%d %H:%M' )

rail_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30686 entries, 0 to 30685
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Payment.Method     30686 non-null  object        
 1   Railcard           10359 non-null  object        
 2   Ticket.Class       30686 non-null  object        
 3   Ticket.Type        30686 non-null  object        
 4   Price              30686 non-null  int64         
 5   Departure.Station  30686 non-null  object        
 6   Arrival.Station    30686 non-null  object        
 7   Departure          30683 non-null  datetime64[ns]
 8   Scheduled.Arrival  30682 non-null  datetime64[ns]
 9   Actual.Arrival     28851 non-null  datetime64[ns]
 10  Journey.Status     30686 non-null  object        
 11  Reason.for.Delay   4121 non-null   object        
 12  Refund.Request     30686 non-null  object        
dtypes: datetime64[ns](3), int64(1), object(9)
memory usage: 3.0+ 

### Inspect the duration series

【说明】计算实际与计划到达时间之差的原始 `timedelta`。

In [342]:

durations = rail_df['Actual.Arrival'] - rail_df['Scheduled.Arrival']
durations.info()


<class 'pandas.core.series.Series'>
RangeIndex: 30686 entries, 0 to 30685
Series name: None
Non-Null Count  Dtype          
--------------  -----          
28850 non-null  timedelta64[ns]
dtypes: timedelta64[ns](1)
memory usage: 239.9 KB


### Create `DelayInMinutes`

【说明】生成 `DelayInMinutes` 变量并展示示例行，是报告中延误分析与后续建模的核心字段。

In [343]:

secs_per_min = 60 
delay_in_minutes = durations.dt.total_seconds() / secs_per_min

rail_df['DelayInMinutes'] = np.where(delay_in_minutes > 0, delay_in_minutes, np.nan)

rail_df.head()


,Payment.Method,Railcard,Ticket.Class,Ticket.Type,Price,Departure.Station,Arrival.Station,Departure,Scheduled.Arrival,Actual.Arrival,Journey.Status,Reason.for.Delay,Refund.Request,DelayInMinutes
0,Contactless,Adult,Standard,Advance,43,London Paddington,Liverpool Lime Street,2024-01-01 11:00:00,2024-01-01 13:30:00,2024-01-01 13:30:00,On Time,NaN,No,NaN
1,Credit Card,Adult,Standard,Advance,23,London Kings Cross,York,2024-01-01 09:45:00,2024-01-01 11:35:00,2024-01-01 11:40:00,Delayed,Signal Failure,No,5.0
2,Credit Card,NaN,Standard,Advance,3,Liverpool Lime Street,Manchester Piccadilly,2024-01-02 18:15:00,2024-01-02 18:45:00,2024-01-02 18:45:00,On Time,NaN,No,NaN
3,Credit Card,NaN,Standard,Advance,13,London Paddington,Reading,2024-01-01 21:30:00,2024-01-01 22:30:00,2024-01-01 22:30:00,On Time,NaN,No,NaN
4,Contactless,NaN,Standard,Advance,76,Liverpool Lime Street,London Euston,2024-01-01 16:45:00,2024-01-01 19:00:00,2024-01-01 19:00:00,On Time,NaN,No,NaN


### On-time counts & NA check

【说明】统计准点/提前旅程的数量以及 `DelayInMinutes` 缺失值，帮助在数据质量与情况说明部分引用。

In [344]:

durations = rail_df['Actual.Arrival'] - rail_df['Scheduled.Arrival']
delay_in_minutes = durations.dt.total_seconds() / 60

# Let's count how many journeys were on-time or arrived early
on_time_or_early_count = (delay_in_minutes <= 0).sum()

print(f"Total number of journeys in the dataset: {len(rail_df)}")
print(f"Number of journeys that were on-time or arrived early: {on_time_or_early_count}")

# Now, let's check for NAs in the column you created.
# I'll use the corrected name 'DelayInMinutes'. If you used another name, please adjust it.
if 'DelayInMinutes' in rail_df.columns:
    na_count = rail_df['DelayInMinutes'].isna().sum()
    print(f"Number of NA values found in the 'DelayInMinutes' column: {na_count}")
else:
    print("Could not find the 'DelayInMinutes' column to check for NAs.")


Total number of journeys in the dataset: 30686
Number of journeys that were on-time or arrived early: 26580
Number of NA values found in the 'DelayInMinutes' column: 28416


## 构造数值特征与 numeric_df

【说明】通过在 DataFrame 中直接创建这些列，可以保证后续派生变量与建模步骤使用同一份数据视图，避免早期人为改名或删除。

In [345]:
rail_df['ScheduledMinutes'] = (rail_df['Scheduled.Arrival'] - rail_df['Departure']).dt.total_seconds() / 60
rail_df['ActualMinutes'] = (rail_df['Actual.Arrival'] - rail_df['Departure']).dt.total_seconds() / 60
rail_df['DepartureHour'] = rail_df['Departure'].dt.hour
rail_df['DepartureDayOfWeek'] = rail_df['Departure'].dt.dayofweek

rail_df[['Price', 'ScheduledMinutes', 'ActualMinutes', 'DepartureHour', 'DepartureDayOfWeek']].head()

,Price,ScheduledMinutes,ActualMinutes,DepartureHour,DepartureDayOfWeek
0,43,150.0,150.0,11.0,0.0
1,23,110.0,115.0,9.0,0.0
2,3,30.0,30.0,18.0,1.0
3,13,60.0,60.0,21.0,0.0
4,76,135.0,135.0,16.0,0.0


【说明】基于上述列构建 `numeric_df`，方便在 pandas 框架下直接做描述统计、协方差与相关分析。

In [346]:
numeric_cols = ['Price', 'ScheduledMinutes', 'ActualMinutes', 'DelayInMinutes', 'DepartureHour', 'DepartureDayOfWeek']
numeric_df = rail_df[numeric_cols].copy()
numeric_df = numeric_df.dropna(subset=['ScheduledMinutes']) 
numeric_df.head()

,Price,ScheduledMinutes,ActualMinutes,DelayInMinutes,DepartureHour,DepartureDayOfWeek
0,43,150.0,150.0,NaN,11.0,0.0
1,23,110.0,115.0,5.0,9.0,0.0
2,3,30.0,30.0,NaN,18.0,1.0
3,13,60.0,60.0,NaN,21.0,0.0
4,76,135.0,135.0,NaN,16.0,0.0


#### Summary stats

【说明】计算上述数值特征的列均值，为报告的数值摘要或描述性统计小节提供指标。

In [347]:

column_means = numeric_df.mean()
print("Column means:\n", column_means)


Column means:
 Price                 23.211050
ScheduledMinutes      70.752934
ActualMinutes         74.064580
DelayInMinutes        42.566079
DepartureHour         11.287484
DepartureDayOfWeek     2.990645
dtype: float64


【说明】在 DelayInMinutes 只对非准点旅程有值的情况下，先筛出所有数值列都非缺失的记录，再用 pandas `.cov()` 计算协方差矩阵，避免成列 `NaN`。

In [348]:

# 为协方差/相关分析准备完全非缺失的子集（DelayInMinutes 只有在非准点旅程中才有值）
numeric_df_complete = numeric_df.dropna()
print(f"可用于协方差/相关矩阵的样本量：{len(numeric_df_complete):,}")

cov_mat = numeric_df_complete.cov()
print("Covariance matrix (pandas, complete cases):\n", cov_mat)


可用于协方差/相关矩阵的样本量：2,270
Covariance matrix (pandas, complete cases):
                           Price  ScheduledMinutes  ActualMinutes  \
Price               2931.426533       1667.048718    1263.694385   
ScheduledMinutes    1667.048718       2312.943834    1745.269841   
ActualMinutes       1263.694385       1745.269841    2363.915631   
DelayInMinutes      -403.354333       -567.673993     618.645790   
DepartureHour        -71.380989        -15.462209     -18.499150   
DepartureDayOfWeek    -4.427898         -2.080627       0.554233   

                    DelayInMinutes  DepartureHour  DepartureDayOfWeek  
Price                  -403.354333     -71.380989           -4.427898  
ScheduledMinutes       -567.673993     -15.462209           -2.080627  
ActualMinutes           618.645790     -18.499150            0.554233  
DelayInMinutes         1186.319783      -3.036940            2.634860  
DepartureHour            -3.036940      19.151098           -0.211015  
DepartureDayOfWeek      

【说明】继续使用上述完整数据子集，通过 pandas `.corr()` 得到可直接引用的皮尔逊相关矩阵。

In [349]:

corr_mat = numeric_df_complete.corr()
print("Correlation matrix (pandas, complete cases):\n", corr_mat)

Correlation matrix (pandas, complete cases):
                        Price  ScheduledMinutes  ActualMinutes  DelayInMinutes  \
Price               1.000000          0.640216       0.480050       -0.216295   
ScheduledMinutes    0.640216          1.000000       0.746387       -0.342701   
ActualMinutes       0.480050          0.746387       1.000000        0.369424   
DelayInMinutes     -0.216295         -0.342701       0.369424        1.000000   
DepartureHour      -0.301263         -0.073467      -0.086944       -0.020148   
DepartureDayOfWeek -0.041826         -0.022126       0.005830        0.039124   

                    DepartureHour  DepartureDayOfWeek  
Price                   -0.301263           -0.041826  
ScheduledMinutes        -0.073467           -0.022126  
ActualMinutes           -0.086944            0.005830  
DelayInMinutes          -0.020148            0.039124  
DepartureHour            1.000000           -0.024661  
DepartureDayOfWeek      -0.024661            1.000

#### Summary stats with pandas

【说明】利用 pandas `describe()` 汇总数值特征统计量，可直接纳入附录或数据描述部分的表格。

In [350]:

numeric_df.describe().T


,count,mean,std,min,25%,50%,75%,max
Price,30680.0,23.211050,29.978043,1.0,5.0,11.0,34.0,267.0
ScheduledMinutes,30680.0,70.752934,36.154801,15.0,30.0,80.0,80.0,260.0
ActualMinutes,28848.0,74.064580,40.195834,15.0,30.0,80.0,90.0,288.0
DelayInMinutes,2270.0,42.566079,34.442993,1.0,19.0,37.0,53.0,180.0
DepartureHour,30680.0,11.287484,5.922240,0.0,6.0,11.0,17.0,23.0
DepartureDayOfWeek,30680.0,2.990645,1.997540,0.0,1.0,3.0,5.0,6.0


## Visualisations

【说明】绘制计划行程时长与票价的散点图，为 EDA 章节讨论票价与行程长度关系提供图像。

In [351]:

# Scatterplot: Scheduled minutes vs Price
plot_df = numeric_df[['ScheduledMinutes', 'Price']].dropna()
plt.figure(figsize=(8,6))
plt.scatter(plot_df['ScheduledMinutes'], plot_df['Price'], s=6, color='darkslateblue')
plt.xlabel('Scheduled travel time (minutes)')
plt.ylabel('Ticket price (£)')
plt.title('Scheduled minutes vs price')
plt.tight_layout()
plt.savefig(FIG_DIR / 'eda_sched_vs_price.pdf', format='pdf')
plt.close()


【说明】绘制实际行程时长与票价的散点图，可在报告中比较计划与实际行程对票价的潜在影响。

In [352]:

# Scatterplot: Actual minutes vs Price
plot_df = numeric_df[['ActualMinutes', 'Price']].dropna()
plt.figure(figsize=(8,6))
plt.scatter(plot_df['ActualMinutes'], plot_df['Price'], s=6, color='indianred', alpha=0.6)
plt.xlabel('Actual travel time (minutes)')
plt.ylabel('Ticket price (£)')
plt.title('Actual minutes vs price')
plt.tight_layout()
plt.savefig(FIG_DIR / 'eda_actual_vs_price.pdf', format='pdf')
plt.close()


【说明】绘制出发小时与票价的散点图，用于说明日内时间与价格的可能关系。

In [353]:

# Scatterplot: Departure hour vs Price
plot_df = numeric_df[['DepartureHour', 'Price']].dropna()
plt.figure(figsize=(8,5))
plt.scatter(plot_df['DepartureHour'], plot_df['Price'], s=4, color='forestgreen', alpha=0.4)
plt.xlabel('Departure hour of day')
plt.ylabel('Ticket price (£)')
plt.title('Departure hour vs price')
plt.tight_layout()
plt.savefig(FIG_DIR / 'eda_hour_vs_price.pdf', format='pdf')
plt.close()


【说明】通过二维直方图展示计划行程时长与票价的联合分布，突出高密度区，服务于 EDA 图表。

In [354]:

# Two-dimensional histogram: scheduled minutes vs price
plot_df = numeric_df[['ScheduledMinutes', 'Price']].dropna()
plt.figure(figsize=(8,6))
plt.hist2d(plot_df['ScheduledMinutes'], plot_df['Price'], bins=60, cmap='Blues')
plt.xlabel('Scheduled travel time (minutes)')
plt.ylabel('Ticket price (£)')
plt.title('2D histogram: scheduled minutes vs price')
cb = plt.colorbar()
cb.set_label('Journey count')
plt.tight_layout()
plt.savefig(FIG_DIR / 'eda_hist2d_sched_price.pdf', format='pdf')
plt.close()


【说明】使用二维 KDE 平滑展示计划行程时长与票价的关系，可在报告里解释主要票价区间。

In [355]:

# Two-dimensional KDE using seaborn (auto-python 也使用 sns.kdeplot)
plot_df = numeric_df[['ScheduledMinutes', 'Price']].dropna()
plt.figure(figsize=(8,6))
sns.kdeplot(x=plot_df['ScheduledMinutes'], y=plot_df['Price'], cmap='Blues', fill=True, thresh=0.05)
plt.xlabel('Scheduled travel time (minutes)')
plt.ylabel('Ticket price (£)')
plt.title('2D KDE: scheduled minutes vs price')
plt.tight_layout()
plt.savefig(FIG_DIR / 'eda_kde_sched_price.pdf', format='pdf')
plt.close()


【说明】绘制计划行程时长、票价与延误的三维散点图，帮助讨论延误与价格/时长的交互模式。

In [356]:

# Three-dimensional scatterplot
plot_df = numeric_df[['ScheduledMinutes', 'Price', 'DelayInMinutes']].dropna()
fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(plot_df['ScheduledMinutes'], plot_df['Price'], plot_df['DelayInMinutes'],
                c=plot_df['DelayInMinutes'], cmap='coolwarm', s=8)
ax.set_xlabel('Scheduled minutes')
ax.set_ylabel('Ticket price (£)')
ax.set_zlabel('Delay minutes (DelayInMinutes)')
ax.set_title('3D scatter: schedule vs price vs delay')
fig.colorbar(sc, ax=ax, shrink=0.6, label='Delay minutes')
plt.tight_layout()
plt.savefig(FIG_DIR / 'eda_scatter3d_sched_price_delay.pdf', format='pdf')
plt.close()


【说明】统计每日旅程数量并绘图，为报告的时间趋势小节提供直接证据。

In [357]:

# Daily journey counts
daily_counts = (rail_df
                .assign(DepartureDate=rail_df['Departure'].dt.date)
                .groupby('DepartureDate')
                .size())
plt.figure(figsize=(12,4))
plt.plot(daily_counts.index, daily_counts.values, color='navy')
plt.xlabel('Date')
plt.ylabel('Journeys')
plt.title('Daily journey volume (Jan–Apr 2024)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(FIG_DIR / 'eda_daily_counts.pdf', format='pdf')
plt.close()


【说明】绘制出发小时分布曲线，描述日内需求峰谷，支撑 EDA 的时间特征分析。

In [358]:

# Departure hour profile
hour_counts = rail_df['Departure'].dt.hour.value_counts().sort_index()
plt.figure(figsize=(8,4))
plt.plot(hour_counts.index, hour_counts.values, marker='o', color='darkorange')
plt.xlabel('Departure hour')
plt.ylabel('Journeys')
plt.title('Hourly departure profile')
plt.xticks(range(0,24,2))
plt.tight_layout()
plt.savefig(FIG_DIR / 'eda_hour_profile.pdf', format='pdf')
plt.close()


【说明】绘制延误分钟的直方图，展示延误程度分布，可用于风险或服务质量讨论。

In [359]:

# Delay distribution for delayed journeys
delay_only = rail_df['DelayInMinutes'].dropna()
plt.figure(figsize=(8,5))
plt.hist(delay_only, bins=40, color='firebrick', alpha=0.7)
plt.xlabel('Delay minutes')
plt.ylabel('Delayed journeys')
plt.title('Distribution of delays (DelayInMinutes)')
plt.tight_layout()
plt.savefig(FIG_DIR / 'eda_delay_hist.pdf', format='pdf')
plt.close()


【说明】绘制 `Journey.Status` 的频次柱状图，展示准点与延误比例，适合放入数据概览。

In [360]:

# Journey status counts
status_counts = rail_df['Journey.Status'].value_counts()
plt.figure(figsize=(7,5))
plt.bar(status_counts.index, status_counts.values, color='lightseagreen')
plt.xlabel('Journey status')
plt.ylabel('Journeys')
plt.title('Journey status distribution')
plt.tight_layout()
plt.savefig(FIG_DIR / 'eda_status_counts.pdf', format='pdf')
plt.close()


【说明】绘制支付方式分布，帮助在报告背景中描述旅客支付偏好。

In [361]:

# Payment method distribution
payment_counts = rail_df['Payment.Method'].value_counts()
plt.figure(figsize=(9,5))
plt.bar(payment_counts.index, payment_counts.values, color='slategray')
plt.xlabel('Payment method')
plt.ylabel('Journeys')
plt.title('Payment method counts')
plt.xticks(rotation=30)
plt.tight_layout()
plt.savefig(FIG_DIR / 'eda_payment_counts.pdf', format='pdf')
plt.close()


【说明】计算并绘制不同票种的退款概率，为票务策略与后续模型的解释提供依据。

In [362]:

# Refund probability by ticket type
refund_binary = (rail_df['Refund.Request'] == 'Yes').astype(int)
refund_rate = (rail_df
              .assign(RefundBinary=refund_binary)
              .groupby('Ticket.Type')['RefundBinary']
              .mean()
              .sort_values(ascending=False))
plt.figure(figsize=(8,5))
plt.bar(refund_rate.index, refund_rate.values, color='mediumpurple')
plt.xlabel('Ticket type')
plt.ylabel('Refund probability')
plt.title('Refund request rate by ticket type')
plt.ylim(0, max(0.05, refund_rate.max()*1.1))
plt.tight_layout()
plt.savefig(FIG_DIR / 'eda_refund_rate_ticket_type.pdf', format='pdf')
plt.close()


## Pair-plots

构建 pair-plot。为保持可读性，如样本数超过 6000，则随机抽样。这里纳入 6 个数值变量，因此图阵为 6×6。

【说明】构建 6×6 的手工配对图，系统展示各数值变量的边际/联合分布，可直接引用于 EDA 附录。

In [363]:

numeric_cols = ['Price','ScheduledMinutes','ActualMinutes','DelayInMinutes','DepartureHour','DepartureDayOfWeek']
plot_df = numeric_df[numeric_cols].dropna()
plot_mat = plot_df.values
if plot_mat.shape[0] > 6000:
    rng = np.random.default_rng(42)
    idx = rng.choice(plot_mat.shape[0], size=6000, replace=False)
    plot_mat = plot_mat[idx]

names = ['Price','Scheduled minutes','Actual minutes','Delay minutes','Departure hour','Departure weekday']
nDataSets = len(names)
plt.figure(figsize=(14,14))
for i in range(nDataSets):
    for j in range(nDataSets):
        plt.subplot(nDataSets, nDataSets, 1 + i + (nDataSets*j))
        if i == j:
            sns.histplot(plot_mat[:,i], stat="density", color = "lightseagreen", alpha=0.6)
            sns.kdeplot(plot_mat[:,i], color = "black")
        else:
            plt.scatter(plot_mat[:,i], plot_mat[:,j], s=4, alpha=0.25, color='tab:blue')
        if j == nDataSets-1:
            plt.xlabel(names[i])
        else:
            plt.xlabel('')
        if i == 0:
            plt.ylabel(names[j])
        else:
            plt.ylabel('')
plt.tight_layout()
plt.savefig(FIG_DIR / 'eda_pairplot_custom.pdf', format='pdf')
plt.close()


## Additional categorical summaries

【说明】定义 `frequency_table` 函数，方便重复生成分类变量的频率与占比表，供附录或正文调用。

In [364]:

def frequency_table(series, top_n=10):
    vc = series.value_counts(dropna=False).head(top_n)
    pct = (vc / series.shape[0] * 100).round(2)
    return pd.DataFrame({'count': vc, 'percent': pct})


【说明】批量生成关键分类变量的频率表，可写入数据描述章节。

In [365]:

categorical_cols = ['Payment.Method', 'Railcard', 'Ticket.Class', 'Ticket.Type',
                    'Journey.Status', 'Refund.Request', 'Reason.for.Delay']
cat_tables = {col: frequency_table(rail_df[col]) for col in categorical_cols}
cat_tables


{'Payment.Method':                 count  percent
 Payment.Method                
 Credit Card     18583    60.56
 Contactless     10423    33.97
 Debit Card       1680     5.47,
 'Railcard':           count  percent
 Railcard                
 NaN       20327    66.24
 Adult      4653    15.16
 Disabled   2923     9.53
 Senior     2783     9.07,
 'Ticket.Class':               count  percent
 Ticket.Class                
 Standard      27724    90.35
 First Class    2962     9.65,
 'Ticket.Type':              count  percent
 Ticket.Type                
 Advance      17039    55.53
 Off-Peak      8307    27.07
 Anytime       5340    17.40,
 'Journey.Status':                 count  percent
 Journey.Status                
 On Time         26565    86.57
 Delayed          2289     7.46
 Cancelled        1832     5.97,
 'Refund.Request':                 count  percent
 Refund.Request                
 No              29591    96.43
 Yes              1095     3.57,
 'Reason.for.Delay':        

【说明】创建 `Journey.Status` × `Refund.Request` 的列联表，支撑对退款行为的讨论。

In [366]:

journey_refund_ct = pd.crosstab(rail_df['Journey.Status'], rail_df['Refund.Request'], normalize='index').round(3)
journey_refund_ct


Refund.Request,No,Yes
Journey.Status,,
Cancelled,0.699,0.301
Delayed,0.762,0.238
On Time,1.000,0.000


【说明】统计最常见的出发-到达组合，帮助在报告中描述旅程结构与热门线路。

In [367]:

route_counts = (rail_df.groupby(['Departure.Station', 'Arrival.Station'])
                       .size()
                       .sort_values(ascending=False)
                       .head(12)
                       .to_frame(name='journeys'))
route_counts


journeys
Departure.Station     Arrival.Station                
Manchester Piccadilly Liverpool Lime Street      4626
London Euston         Birmingham New Street      4109
London Kings Cross    York                       3731
London Paddington     Reading                    3728
London St Pancras     Birmingham New Street      3436
Liverpool Lime Street Manchester Piccadilly      2918
                      London Euston              1080
Birmingham New Street London St Pancras           701
London Euston         Manchester Piccadilly       598
London Paddington     Oxford                      485
London St Pancras     Leicester                   337
Manchester Piccadilly London Euston               313

## 问题四：MediumPrice 单变量逻辑回归


本节只针对非准点旅程，构造 `MediumPrice` 与 `RefundBinary` 两个变量，使用单变量逻辑回归评估票价所在区间对退款概率的影响，并计算 £5 与 £25 两个场景下的预测概率。

### 4.1 筛选非准点样本

【说明】筛掉 `Journey.Status == 'On Time'` 的旅程，只保留晚点或取消的样本，为本题所需的建模范围。

In [368]:

late_df = rail_df[rail_df['Journey.Status'] != 'On Time'].copy()
late_df = late_df.dropna(subset=['Refund.Request'])  # 删除极少数缺失退款状态的记录
print(f"非准点样本行数：{len(late_df):,}")
late_df[['Journey.Status', 'Price', 'Refund.Request']].head()


非准点样本行数：4,121


,Journey.Status,Price,Refund.Request
1,Delayed,23,No
8,Delayed,37,No
20,Delayed,7,Yes
26,Delayed,34,Yes
38,Cancelled,7,No


### 4.2 构造 RefundBinary 与 MediumPrice

【说明】在过滤后的数据集中新增逻辑回归所需的两个变量：`RefundBinary` 表示是否退款，`MediumPrice` 表示票价是否处在 (10, 30] 区间。

In [369]:

# RefundBinary: Yes -> 1, 其余 -> 0
late_df['RefundBinary'] = np.where(late_df['Refund.Request'].str.strip().str.lower() == 'yes', 1, 0)

# MediumPrice: 指示价格是否落在 10 < Price <= 30 区间
late_df['MediumPrice'] = np.where((late_df['Price'] > 10) & (late_df['Price'] <= 30), 1, 0)

late_df[['Price', 'MediumPrice', 'Refund.Request', 'RefundBinary']].head()


,Price,MediumPrice,Refund.Request,RefundBinary
1,23,1,No,0
8,37,0,No,0
20,7,0,Yes,1
26,34,0,Yes,1
38,7,0,No,0


### 4.3 MediumPrice × RefundBinary 交叉表

【说明】输出 MediumPrice 与 RefundBinary 的列联表及退款比例，便于在报告中描述两者的初步关系。

In [370]:

mp_refund_ct = pd.crosstab(late_df['MediumPrice'], late_df['RefundBinary'], margins=True)
mp_refund_rate = (mp_refund_ct.div(mp_refund_ct['All'], axis=0)).round(3)
mp_refund_ct, mp_refund_rate


(RefundBinary     0     1   All
 MediumPrice                   
 0             2528   855  3383
 1              498   240   738
 All           3026  1095  4121,
 RefundBinary      0      1  All
 MediumPrice                    
 0             0.747  0.253  1.0
 1             0.675  0.325  1.0
 All           0.734  0.266  1.0)

### 4.4 拟合单变量逻辑回归

【说明】使用 statsmodels 的 Logit 函数，以 `RefundBinary` 为响应、`MediumPrice` 为唯一解释变量拟合逻辑回归，并输出模型概要。

In [371]:

import statsmodels.api as sm

X = sm.add_constant(late_df['MediumPrice'])  # 添加截距项
logit_model = sm.Logit(late_df['RefundBinary'], X)
logit_result = logit_model.fit(disp=False)
logit_result.summary2()


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Method:           MLE       
Dependent Variable: RefundBinary     Pseudo R-squared: 0.003     
Date:               2025-11-13 23:10 AIC:              4759.9061 
No. Observations:   4121             BIC:              4772.5538 
Df Model:           1                Log-Likelihood:   -2378.0   
Df Residuals:       4119             LL-Null:          -2385.8   
Converged:          1.0000           LLR p-value:      7.1240e-05
No. Iterations:     5.0000           Scale:            1.0000    
------------------------------------------------------------------
              Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
------------------------------------------------------------------
const        -1.0841    0.0396  -27.4020  0.0000  -1.1616  -1.0065
MediumPrice   0.3541    0.0880    4.0252  0.0001   0.1817   0.5266
=================================================================

"""

### 4.5 系数与赔率比解释

【说明】整理模型系数、标准误、p 值与赔率比，供报告撰写时引用。

In [372]:

coef_table = pd.DataFrame({
    'Coefficient': logit_result.params,
    'Std.Err': logit_result.bse,
    'p-value': logit_result.pvalues,
    'Odds Ratio': np.exp(logit_result.params)
})
coef_table


,Coefficient,Std.Err,p-value,Odds Ratio
const,-1.084082,0.039562,2.593479e-165,0.338212
MediumPrice,0.354121,0.087976,5.693172e-05,1.424928


### 4.6 计算 £5 与 £25 的退款概率

【说明】根据逻辑回归系数，分别将 MediumPrice=0（£5）和 MediumPrice=1（£25）代入模型，得到对应的退款概率。

In [373]:

# 简单的 sigmoid 函数用于将 log-odds 转换为概率
sigmoid = lambda z: 1 / (1 + np.exp(-z))

beta0 = logit_result.params['const']
beta1 = logit_result.params['MediumPrice']

p5 = sigmoid(beta0)               # Price=5 -> MediumPrice=0
p25 = sigmoid(beta0 + beta1)      # Price=25 -> MediumPrice=1

prob_table = pd.DataFrame({
    'Price (£)': [5, 25],
    'MediumPrice': [0, 1],
    'Predicted Refund Probability': [p5, p25]
})
prob_table.round({'Predicted Refund Probability': 4})


,Price (£),MediumPrice,Predicted Refund Probability
0,5,0,0.2527
1,25,1,0.3252


## 问题五：基于多变量的退款概率建模


本节在清洗后的 `rail_df` 上构建多变量分类模型来预测 `Refund.Request` 是否为 "Yes"，并比较多种模型的表现，最终选出一个模型对 `ToPredict.csv` 中的 8 条记录输出退款概率。

### 5.1 构造响应变量与特征列

【说明】根据题目要求，创建 `RefundBinary`、`DelayInMinutes_filled` 等建模变量，并选定用于建模的数值与分类特征。

In [374]:

rail_df['RefundBinary'] = (rail_df['Refund.Request'] == 'Yes').astype(int)
rail_df['DelayInMinutes_filled'] = rail_df['DelayInMinutes'].fillna(0)

feature_cols = [
    'Price',
    'DelayInMinutes_filled',
    'Journey.Status',
    'Ticket.Type',
    'Ticket.Class',
    'Payment.Method',
    'Railcard',
    'DepartureHour',
    'DepartureDayOfWeek'
]

X_raw = rail_df[feature_cols].copy()
y = rail_df['RefundBinary'].copy()

categorical_cols = ['Journey.Status', 'Ticket.Type', 'Ticket.Class', 'Payment.Method', 'Railcard']
X_raw[categorical_cols] = X_raw[categorical_cols].fillna('Unknown')
X_raw['DepartureHour'] = X_raw['DepartureHour'].fillna(X_raw['DepartureHour'].median())
X_raw['DepartureDayOfWeek'] = X_raw['DepartureDayOfWeek'].fillna(X_raw['DepartureDayOfWeek'].median())

X = pd.get_dummies(
    X_raw,
    columns=categorical_cols,
    drop_first=True
)

print(f"特征矩阵维度：{X.shape}")
print(f"退款申请占比：{y.mean():.3f}")


特征矩阵维度：(30686, 14)
退款申请占比：0.036


### 5.2 训练/测试集划分

【说明】使用 80/20 分层抽样划分训练和测试集，确保退款与否的比例保持一致。

In [375]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=123,
    stratify=y
)
print(f"训练集：{X_train.shape[0]} 行，测试集：{X_test.shape[0]} 行")


训练集：24548 行，测试集：6138 行


### 5.3 训练多种分类模型

【说明】拟合 Logistic 回归、浅层决策树与随机森林三个模型，以比较性能与可解释性。所有模型均使用 class_weight='balanced' 来处理退款申请的类别不平衡问题。

In [376]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 使用 class_weight='balanced' 处理类别不平衡（退款申请仅占3.57%）
logit_model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=123)
logit_model.fit(X_train, y_train)

tree_model = DecisionTreeClassifier(max_depth=4, class_weight='balanced', random_state=123)
tree_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=123, n_jobs=-1)
rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=123)

### 5.4 测试集表现对比

【说明】计算 Accuracy、Precision、Recall、F1、ROC-AUC 与 PR-AUC，方便在报告中引用。PR-AUC 对类别不平衡数据尤其重要。

In [377]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score

def evaluate_model(name, model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    return {
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, zero_division=0),
        'Recall': recall_score(y_test, y_pred, zero_division=0),
        'F1': f1_score(y_test, y_pred, zero_division=0),
        'ROC-AUC': roc_auc_score(y_test, y_proba),
        'PR-AUC': average_precision_score(y_test, y_proba)
    }

results = [
    evaluate_model('Logistic regression', logit_model, X_test, y_test),
    evaluate_model('Decision tree', tree_model, X_test, y_test),
    evaluate_model('Random forest', rf_model, X_test, y_test)
]

results_df = pd.DataFrame(results).round(3)
results_df

,Model,Accuracy,Precision,Recall,F1,ROC-AUC,PR-AUC
0,Logistic regression,0.941,0.375,0.973,0.541,0.984,0.669
1,Decision tree,0.930,0.338,0.995,0.505,0.986,0.649
2,Random forest,0.970,0.569,0.662,0.612,0.967,0.745


【说明】绘制三个模型的ROC曲线对比图，直观展示各模型在不同分类阈值下的真阳性率与假阳性率权衡，为模型选择提供可视化依据。

In [378]:
# ROC curves comparison
from sklearn.metrics import roc_curve, auc

models_to_plot = {
    'Logistic regression': logit_model,
    'Decision tree': tree_model,
    'Random forest': rf_model
}

colors = ['steelblue', 'forestgreen', 'coral']

plt.figure(figsize=(10, 8))

for (name, model), color in zip(models_to_plot.items(), colors):
    y_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color=color, lw=2.5, label=f'{name} (AUC = {roc_auc:.3f})')

# Baseline: random guessing
plt.plot([0, 1], [0, 1], color='darkgray', lw=2, linestyle='--', label='Random guess (AUC = 0.500)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves: Model Comparison for Refund Prediction', fontsize=13)
plt.legend(loc='lower right', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(FIG_DIR / 'model_roc_comparison.pdf', format='pdf')
plt.close()

【说明】绘制Precision-Recall曲线对比图。由于退款申请占比仅3.57%，类别严重不平衡，PR曲线比ROC曲线能更准确地反映模型在少数类（退款）上的表现。

In [379]:
# Precision-Recall curves comparison
from sklearn.metrics import precision_recall_curve, average_precision_score

plt.figure(figsize=(10, 8))

for (name, model), color in zip(models_to_plot.items(), colors):
    y_proba = model.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_proba)
    pr_auc = average_precision_score(y_test, y_proba)
    plt.plot(recall, precision, color=color, lw=2.5, label=f'{name} (AP = {pr_auc:.3f})')

# Baseline: random guessing for imbalanced data
baseline = y_test.mean()
plt.plot([0, 1], [baseline, baseline], color='darkgray', lw=2, linestyle='--', 
         label=f'Random guess (AP = {baseline:.3f})')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Precision-Recall Curves: Model Comparison for Refund Prediction', fontsize=13)
plt.legend(loc='best', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(FIG_DIR / 'model_pr_comparison.pdf', format='pdf')
plt.close()

### 5.5 选择最终模型

【说明】综合性能与可解释性，自动选择最优模型；若随机森林优于 Logit 很多，就选它，否则偏好 Logit。

In [380]:
model_map = {
    'Logistic regression': logit_model,
    'Decision tree': tree_model,
    'Random forest': rf_model
}

best_by_auc = results_df.loc[results_df['ROC-AUC'].idxmax()]
logit_auc = results_df.loc[results_df['Model'] == 'Logistic regression', 'ROC-AUC'].iloc[0]

if (best_by_auc['Model'] != 'Logistic regression') and (best_by_auc['ROC-AUC'] - logit_auc) > 0.01:
    final_model_name = best_by_auc['Model']
    final_model = model_map[final_model_name]
    final_model_rationale = f"{final_model_name} achieves the highest ROC AUC and F1, so it is selected as the final model."
else:
    final_model_name = 'Logistic regression'
    final_model = logit_model
    final_model_rationale = "Logistic regression offers comparable ROC AUC while providing clear interpretability, so it is selected as the final model."

final_feature_columns = X.columns.tolist()
print(f"最终模型：{final_model_name}")
print(final_model_rationale)


最终模型：Logistic regression
Logistic regression offers comparable ROC AUC while providing clear interpretability, so it is selected as the final model.


### 5.6 预测 ToPredict.csv 的退款概率

【说明】对 `ToPredict.csv` 执行同样的特征工程并用最终模型给出退款概率，供报告引用。

In [381]:

df_pred = pd.read_csv('ToPredict.csv')

# 确保存在必要的时间与延误字段
time_cols = ['Departure', 'Scheduled.Arrival', 'Actual.Arrival']
for col in time_cols:
    if col in df_pred.columns:
        df_pred[col] = pd.to_datetime(df_pred[col], format='%Y-%m-%d %H:%M', errors='coerce')

if 'DelayInMinutes' not in df_pred.columns:
    delay_minutes = (df_pred['Actual.Arrival'] - df_pred['Scheduled.Arrival']).dt.total_seconds() / 60
    df_pred['DelayInMinutes'] = delay_minutes.where(delay_minutes > 0)

if 'DepartureHour' not in df_pred.columns:
    df_pred['DepartureHour'] = df_pred['Departure'].dt.hour
if 'DepartureDayOfWeek' not in df_pred.columns:
    df_pred['DepartureDayOfWeek'] = df_pred['Departure'].dt.dayofweek

# 先构造与训练数据一致的填补列
if 'DelayInMinutes_filled' not in df_pred.columns:
    df_pred['DelayInMinutes_filled'] = df_pred['DelayInMinutes'].fillna(0)
else:
    df_pred['DelayInMinutes_filled'] = df_pred['DelayInMinutes_filled'].fillna(0)

df_pred['DepartureHour'] = df_pred['DepartureHour'].fillna(df_pred['DepartureHour'].median())
df_pred['DepartureDayOfWeek'] = df_pred['DepartureDayOfWeek'].fillna(df_pred['DepartureDayOfWeek'].median())

missing_cols = set(feature_cols) - set(df_pred.columns)
if missing_cols:
    raise ValueError(f'ToPredict 缺少以下列，需要先补齐: {missing_cols}')

X_pred_raw = df_pred[feature_cols].copy()

X_pred = pd.get_dummies(
    X_pred_raw,
    columns=['Journey.Status', 'Ticket.Type', 'Ticket.Class', 'Payment.Method', 'Railcard'],
    drop_first=True
)

for col in final_feature_columns:
    if col not in X_pred.columns:
        X_pred[col] = 0
X_pred = X_pred[final_feature_columns]

pred_proba = final_model.predict_proba(X_pred)[:, 1]
df_pred['PredRefundProb'] = pred_proba

pred_cols = ['Price', 'Journey.Status', 'Ticket.Type', 'Ticket.Class', 'Payment.Method', 'PredRefundProb']
print("ToPredict 退款概率：")
df_pred[pred_cols].round({'PredRefundProb': 4})


ToPredict 退款概率：


,Price,Journey.Status,Ticket.Type,Ticket.Class,Payment.Method,PredRefundProb
0,54,On Time,Advance,First Class,Debit Card,0.0217
1,7,On Time,Advance,Standard,Credit Card,0.0010
2,113,Delayed,Off-Peak,Standard,Debit Card,0.9930
3,3,Delayed,Off-Peak,Standard,Contactless,0.8893
4,4,Cancelled,Off-Peak,Standard,Credit Card,0.9761
5,3,Cancelled,Advance,Standard,Contactless,0.9673
6,126,Delayed,Off-Peak,Standard,Debit Card,0.9941
7,22,Cancelled,Advance,Standard,Credit Card,0.9718


### 5.7 输出汇总表

【说明】整理模型评估结果与 ToPredict 预测结果，方便直接复制到报告。

In [383]:

print("模型测试表现 (Test Metrics):")
display(results_df)

print("ToPredict 退款概率：")
pred_cols_display = ['Price', 'Journey.Status', 'Ticket.Type', 'Ticket.Class', 'Payment.Method', 'PredRefundProb']
missing_cols = [col for col in pred_cols_display if col not in df_pred.columns]
if missing_cols:
    raise ValueError(f"ToPredict 结果缺少列：{missing_cols}. 请先运行第 5.6 单元。")
display(df_pred[pred_cols_display].round({'PredRefundProb': 4}))


模型测试表现 (Test Metrics):


,Model,Accuracy,Precision,Recall,F1,ROC-AUC,PR-AUC
0,Logistic regression,0.941,0.375,0.973,0.541,0.984,0.669
1,Decision tree,0.930,0.338,0.995,0.505,0.986,0.649
2,Random forest,0.970,0.569,0.662,0.612,0.967,0.745


ToPredict 退款概率：


,Price,Journey.Status,Ticket.Type,Ticket.Class,Payment.Method,PredRefundProb
0,54,On Time,Advance,First Class,Debit Card,0.0217
1,7,On Time,Advance,Standard,Credit Card,0.0010
2,113,Delayed,Off-Peak,Standard,Debit Card,0.9930
3,3,Delayed,Off-Peak,Standard,Contactless,0.8893
4,4,Cancelled,Off-Peak,Standard,Credit Card,0.9761
5,3,Cancelled,Advance,Standard,Contactless,0.9673
6,126,Delayed,Off-Peak,Standard,Debit Card,0.9941
7,22,Cancelled,Advance,Standard,Credit Card,0.9718
